In [ ]:
# Hit
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct 30 16:43:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [ ]:
# Hit
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer

NotImplementedError: ignored

In [ ]:
# Hit
from huggingface_hub import notebook_login

notebook_login()


Then you need to install Git-LFS to upload your model checkpoints:

In [ ]:
# Hit
%%capture
!apt install git-lfs

## Prepare Data, Tokenizer, Feature Extractor

### Create Wav2Vec2CTCTokenizer

Let's start by loading the dataset and taking a look at its structure.

In [ ]:
# Hit
from datasets import load_dataset, load_metric

In [ ]:
data_files={"train":"/content/train_data.csv","test":"/content/test_data.csv"}
csv_dataset= load_dataset("csv",data_files=data_files)
csv_dataset

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-841ccd53a16a7810/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


/usr/local/lib/python3.10/dist-packages/datasets/packaged_modules/csv/csv.py:155: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/usr/local/lib/python3.10/dist-packages/datasets/packaged_modules/csv/csv.py:155: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio_filepath', 'text'],
        num_rows: 5320
    })
    test: Dataset({
        features: ['audio_filepath', 'text'],
        num_rows: 1336
    })
})

In [ ]:
# Hit
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(csv_dataset["train"].remove_columns(["audio_filepath"]), num_examples=10)

,text
0,I think the education system in our state has aged like milk
1,so because when I have to make payment I do it by Apple Pay
2,No
3,In the academic year 2020 - 2021 Can you list all the AICTE approved colleges offering Diploma level programs in Management ?
4,provide me the world latest news
5,wherever you go in the world you can communicate in English
6,"You can select the seat you want. By using cutting-edge technology like mobile boarding passes, companies are able to cut their expenditure and time, which ultimately benefits the travelers"
7,"nowadays we have Arijit, Sonu Nigam. We try to copy unconsciously, we try to copy them."
8,"Supplemental Nutrition Assistance Program, SNAP"
9,Off


In [ ]:
import re
# Hit
# chars_to_ignore_regex = '[^A-Za-z]+'

def remove_special_characters(batch):
    batch["text"] = re.sub(r'[^A-Za-z\s]+', '', batch["text"]).lower() + " "
    return batch

In [ ]:
csv_dataset = csv_dataset.map(remove_special_characters)

0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [ ]:
show_random_elements(csv_dataset["train"].remove_columns(["audio_filepath"]))

,text
0,available in beed district
1,i want to order eggs can you show me what all is available in that category
2,but sometimes during the busiest you know booking will be getting delayed
3,being hurt badly in hockey
4,yes i have traveled many places with a large group of friends and family as well all the trip was fun and thrilling and a unique kind of experience
5,can you delete chings paneer chilli masala and borges vegan natura rice almond drink from my shopping cart in addition to this please remove bottles beardo onion oil concentrate
6,stop
7,how many types of pizza items items are available
8,to get a good education in my district because i work in a college and i see that because of that program
9,i want to get enrolled myself in an institute offering programs in management


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = csv_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=csv_dataset.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [ ]:
len(vocabs["train"]["vocab"][0])

29

In [ ]:
len(vocabs["test"]["vocab"][0])

27

In [ ]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'\r': 0,
 'e': 1,
 'b': 2,
 'o': 3,
 'n': 4,
 't': 5,
 'h': 6,
 'y': 7,
 'x': 8,
 'l': 9,
 ' ': 10,
 'm': 11,
 'f': 12,
 'r': 13,
 'a': 14,
 'z': 15,
 '\n': 16,
 'd': 17,
 'c': 18,
 'i': 19,
 'g': 20,
 'w': 21,
 's': 22,
 'q': 23,
 'v': 24,
 'k': 25,
 'u': 26,
 'p': 27,
 'j': 28}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

31

In [ ]:
vocab_dict

{'\r': 0,
 'e': 1,
 'b': 2,
 'o': 3,
 'n': 4,
 't': 5,
 'h': 6,
 'y': 7,
 'x': 8,
 'l': 9,
 'm': 11,
 'f': 12,
 'r': 13,
 'a': 14,
 'z': 15,
 '\n': 16,
 'd': 17,
 'c': 18,
 'i': 19,
 'g': 20,
 'w': 21,
 's': 22,
 'q': 23,
 'v': 24,
 'k': 25,
 'u': 26,
 'p': 27,
 'j': 28,
 '|': 10,
 '[UNK]': 29,
 '[PAD]': 30}

Cool, now our vocabulary is complete and consists of 30 tokens, which means that the linear layer that we will add on top of the pretrained Wav2Vec2 checkpoint will have an output dimension of 30.

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [ ]:
# Hit
import json

In [ ]:
# Hit
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
# Hit
repo_name = "/content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2"

### Create Wav2Vec2 Feature Extractor

A Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, models should **always** make use of the `attention_mask` to mask padded tokens. However, due to a very specific design choice of `Wav2Vec2`'s "base" checkpoint, better results are achieved when using no `attention_mask`. This is **not** recommended for other speech models. For more information, one can take a look at [this](https://github.com/pytorch/fairseq/issues/3227) issue. **Important** If you want to use this notebook to fine-tune [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60), this parameter should be set to `True`.

In [ ]:
# Hit
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

Great, Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [ ]:
# Hit
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just the transcription. In addition to `'text'`, our datasets include two more column names `'file'` and `'audio'`. `'file'` states the absolute path of the audio file. Let's take a look.

In [ ]:
csv_dataset["train"][0]["audio_filepath"]

'audio/281474976884635_f3269_chunk_0.wav'

`Wav2Vec2` expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.

 Thankfully, `datasets` does this automatically when calling the column `audio`. Let try it out.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Hit
path="/content/drive/MyDrive/ANLP_PROJECT/"

In [ ]:
csv_dataset.num_rows

{'train': 5320, 'test': 1336}

In [ ]:
csv_dataset

DatasetDict({
    train: Dataset({
        features: ['audio_filepath', 'text'],
        num_rows: 5320
    })
    test: Dataset({
        features: ['audio_filepath', 'text'],
        num_rows: 1336
    })
})

In [ ]:
path+csv_dataset["train"][0]["audio_filepath"]

'/content/drive/MyDrive/ANLP_PROJECT/audio/281474976884635_f3269_chunk_0.wav'

In [ ]:
rand_int = random.randint(0, len(csv_dataset["train"]))

print("Target text:", csv_dataset["train"][rand_int]["text"])
audio_file_path = path+csv_dataset["train"][rand_int]["audio_filepath"]
audio_data, sampling_rate = librosa.load(audio_file_path, sr=16000)
print("Input array shape:", np.asarray(audio_data).shape)
print("Sampling rate:" ,sampling_rate)
# print("Sampling rate:", ["train"][rand_int]["audio"]["sampling_rate"])

Target text: banks are regulated by the federal government and sometimes state governments to try to keep to them from taking on too much risk and imperiling the economy 


NameError: ignored

Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can process the dataset to the format expected by the model for training. We will make use of the `map(...)` function.

First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_values` from the loaded audio file. In our case, the `Wav2Vec2Processor` only normalizes the data. For other speech models, however, this step can include more complex feature extraction, such as [Log-Mel feature extraction](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum).
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [ ]:
# Hit
import librosa

In [ ]:
def prepare_dataset(batch):
    audio_path = path+batch["audio_filepath"]
    audio_data, sampling_rate = librosa.load(audio_path, sr=16000)


    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio_data, sampling_rate=16000).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    # print(batch)
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

Let's apply the data preparation function to all examples.

In [ ]:
csv_dataset = csv_dataset.map(prepare_dataset, remove_columns=csv_dataset.column_names["train"], num_proc=4)

In [ ]:
csv_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 5320
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 1336
    })
})

In [ ]:
for i in range(0,81):
  print(csv_dataset['train'][i]['input_length'])

In [ ]:
csv_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 5320
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 1336
    })
})

Long input sequences require a lot of memory. Since `Wav2Vec2` is based on `self-attention` the memory requirement scales quadratically with the input length for long input sequences (*cf.* with [this](https://www.reddit.com/r/MachineLearning/comments/genjvb/d_why_is_the_maximum_input_sequence_length_of/) reddit post). For this demo, let's filter all sequences that are longer than 4 seconds out of the training dataset.

In [ ]:
max_input_length_in_sec = 4.0
csv_dataset["train"] = csv_dataset["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/6 [00:00<?, ?ba/s]

Awesome, now we are ready to start training!

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
# Hit
import torch

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
# Hit
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
# Hit
import numpy as np

In [ ]:
pred_str=processor.batch_decode([1,2])

In [ ]:
processor.tokenizer.pad_token_id

30

In [ ]:
def compute_metrics(pred):
    # print(pred)
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    # print(pred_ids)
    # print(pred_ids)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred.label_ids[pred.label_ids>30]=processor.tokenizer.unk_token_id

    pred_str = processor.batch_decode(pred_ids)
    print(len(pred_str))

    for i in range(0,len(pred_str)):
      if(pred_str[i]==''):
        pred_str[i]="UNK"



    # print(len(pr_str))|
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    print(pred_str)
    for i in range(0,len(label_str)):
      if(label_str[i]==''):
        label_str[i]="UNK"
    for i in label_str:
      if(i==''):
          print("blank")
    print(len(label_str))
    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:356: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'project_q.bias', 'project_q.weight', 'project_hid.weight', 'quantizer.weight_proj.bias', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
model.freeze_feature_encoder()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=csv_dataset["train"],
    eval_dataset=csv_dataset["test"],
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend




---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take between 90 and 270 minutes depending on the GPU allocated to this notebook. While the trained model yields satisfying results on *Timit*'s test data, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how Wav2Vec2's [base](https://huggingface.co/facebook/wav2vec2-base), [large](https://huggingface.co/facebook/wav2vec2-large), and [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60) checkpoints can be fine-tuned on any English dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

Depending on what GPU was allocated to your google colab it might be possible that you are seeing an `"out-of-memory"` error here. In this case, it's probably best to reduce `per_device_train_batch_size` to 16 or even less and eventually make use of [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments).

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2516
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9450


Step,Training Loss,Validation Loss,Wer
500,6.076300,7.030796,0.998894
1000,2.445900,2.041698,0.849646
1500,1.425600,1.992177,0.698998
2000,1.056700,6.460894,0.563458
2500,0.857700,1.452981,0.607827
3000,0.697100,1.810960,0.522553
3500,0.631300,1.472654,0.506191
4000,0.517400,1.238012,0.472140
4500,0.498200,1.259134,0.487765
5000,0.409200,1.422748,0.479953


The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1336
  Batch size = 8


1336
['UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 'UNK', 

Saving model checkpoint to /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-500
Configuration saved in /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-500/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-500/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1336
  Batch size = 8


...

1336
['and after that we celebrate toyceala say', 'so i think these two enlish government initiatives have healphd people from my district as wellas from the state', 'and you would like to cahchyour dhose no maks', 'language the amaints of communiclation is nactualally the exprestion or representatio', 'people visiting ur statefrom different cuntries and people ead up to these arts and crafs', 'hoah', 'oan', 'therepor there is no to mynolis there is no comon hisory shiarey history of the district', 'a anathal pradeshihas ahmong', 'what is the name of sa', 'and otp has been sent to phon numbar n t  t   jiropleje intarit phr donloating the vaccination saptificlre for wan isry names shahnor', 'ter are lots of region ther e lots of sence that i think how the es sports can', 'find the taop newtual fund skchienmes wated by chradable agencies such as bhajhas finance', 'teana of garagjha like', 'also we love reaciycling or uppsyceling r oll yutencels wasels or plastic payened buckets as our pl

Saving model checkpoint to /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-8500
Configuration saved in /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-8500/config.json
Model weights saved in /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-8500/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-8500/preprocessor_config.json
Deleting older checkpoint [/content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-7500] due to args.save_total_limit
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1336
  Batch size = 8


1336
['and after thatwe celebrate toypcualasay', 'so i think these two enlish government initiatives have helthd people from my district as well asfrom the state', 'and you would like to cathchyour dhose no maks', 'language theh a mants of communiculation is nactualally the exprestion or representatio', 'people visiting r statefrom different cuntries and people ead up to these arts and crafs', 'hoah', 'oan', 'therepour there is no to mynolis there is no comon hisory siare history of the district', 'a anathal pradeashihas ahmong', 'what is the name of sa', 'and otp has been sent to phon nmbar n t  t n  jiropleje intarit phr donloadding the vaccination sartificere for win isri names shahnor', 'ter are lots of region ther e lots of seincs that i think how thes sports can', 'find the tapk neuwtualfund schienmes wated by chreadable agancies suchas bhajas finance', 'tean of garajhea like', 'also we love reciycling or uppsyceling r oll utencels weasels or plastic payene buckets as our planter

Saving model checkpoint to /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-9000
Configuration saved in /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-9000/config.json
Model weights saved in /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-9000/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-9000/preprocessor_config.json
Deleting older checkpoint [/content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-8000] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9450, training_loss=0.8918780897029494, metrics={'train_runtime': 4610.6243, 'train_samples_per_second': 16.371, 'train_steps_per_second': 2.05, 'total_flos': 1.2603744826596173e+18, 'train_loss': 0.8918780897029494, 'epoch': 30.0})

In [ ]:
trainer.save_model("we2vec2_fine_tuned_model")

Saving model checkpoint to we2vec2_fine_tuned_model
Configuration saved in we2vec2_fine_tuned_model/config.json
Model weights saved in we2vec2_fine_tuned_model/pytorch_model.bin
Feature extractor saved in we2vec2_fine_tuned_model/preprocessor_config.json


```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
```

In [ ]:
# Hit
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("/content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-9000").cuda()
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-9000")

Some weights of the model checkpoint at /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-9000 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at /content/drive/MyDrive/ANLP_PROJECT/rishav_wev2vec2/checkpoint-9000 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embe

### Evaluate

In the final part, we run our model on some of the validation data to get a feeling for how well it works.

Let's load the `processor` and `model`.

Now, we will make use of the `map(...)` function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary `"results"`.

**Note**: we evaluate the test data set with `batch_size=1` on purpose due to this [issue](https://github.com/pytorch/fairseq/issues/3227). Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [ ]:
import torch

In [ ]:
with torch.no_grad():
  # /content/Recording (4).m4a
    audio_data, sampling_rate = librosa.load('/content/Turn_on_the light.m4a', sr=16000)
    input_values = torch.tensor(audio_data, device="cuda").unsqueeze(0)
    logits = model(input_values).logits
pred_ids = torch.argmax(logits, dim=-1)
# pred_text = processor.batch_decode(pred_ids)[0]
pred_text = tokenizer.batch_decode(pred_ids)[0]
tokens = [token for token in pred_text.split("[PAD]") if token.strip()]

# Join the tokens to create a single string
resulting_string = " ".join(tokens)
resulting_string

<ipython-input-26-e8d81f74bedf>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sampling_rate = librosa.load('/content/Turn_on_the light.m4a', sr=16000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


'turn onthe like'

In [ ]:
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root

In [ ]:
name = "meta-llama/Llama-2-7b-chat-hf"
text = "Hello my name is"
max_new_tokens = 20

def generate_from_model(model, tokenizer):
  encoded_input = tokenizer(text, return_tensors='pt')
  output_sequences = model.generate(input_ids=encoded_input['input_ids'].cuda())
  return tokenizer.decode(output_sequences[0], skip_special_tokens=True)

In [ ]:
!pip install --upgrade transformers


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 14.3 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

pipe = pipeline(model=name, model_kwargs= {"device_map": "auto", "load_in_8bit": True}, max_new_tokens=max_new_tokens)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe("i can correct this sentence '"+resulting_string+"' as")

[{'generated_text': "i can correct this sentence 'turn onthe like' as 'turn on the lights' and 'can correct' as 'can correct this sentence'\n\n"}]

In [ ]:
# Here we will build a function which will take a string as input
location = ['living' , 'kitchen','bathroom','bed']
operation=['off','on']
appliance=['bulb','fan']
rows,cols=(len(appliance),len(location))
status=[[0 for i in range(cols)] for j in range(rows)]
def home_automation(prompt):
  location_index , operation_index ,appliance_index=(-1,-1,-1)
  location_count , operation_count , appliance_count=(0,0,0)
  for word in prompt.split():
    if(word in location):
      location_index = location.index(word)
      location_count=location_count+1
    if(word in operation):
      operation_index = operation.index(word)
      operation_count = operation_count+1
    if(word in appliance):
      appliance_index= appliance.index(word)
      appliance_count = appliance_count+1
  if(location_count==1 and operation_count==1 and appliance_count==1):
    if(operation_index==0):
      if(status[appliance_index][location_index]==0):
        print("It is already switched off")
        return
      else:
        status[appliance_index][location_index]=0
    elif(operation_index==1):
      if(status[appliance_index][location_index]==1):
        print("It is already switched on")
        return
      else:
        status[appliance_index][location_index]=1
    print("Switching "+operation[operation_index]+" the "+appliance[appliance_index]+" of "+location[location_index]+" . ")
  else:
    print("Please enter again")

In [ ]:
home_automation("Turn off the fan of the kitchen")

In [ ]:
status

In [ ]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)

  return batch

In [ ]:
results = csv_dataset["test"].map(map_to_result, remove_columns=csv_dataset["test"].column_names)

0ex [00:00, ?ex/s]

Let's compute the overall WER now.

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

ValueError: ignored

In [ ]:
show_random_elements(results)

,pred_str,text
0,[PAD]l[PAD]earn[PAD] [PAD],learn
1,c[PAD]or[PAD]d[PAD]ing [PAD]t[PAD]o[PAD] [PAD]re[PAD]p[PAD]ort[PAD]s[PAD] [PAD]it[PAD] [PAD]w[PAD]as[PAD] [PAD]a[PAD] [PAD]sh[PAD]o[PAD]s[PAD]i[PAD]d a[PAD]t[PAD]ack,according to reports it was a suicide attack
2,[PAD]a[PAD]b[PAD]out[PAD] [PAD] [PAD]w[PAD]ords [PAD]are [PAD]ad[PAD]d[PAD]ed t[PAD]o the [PAD]v[PAD]ic[PAD]t[PAD]or[PAD]r[PAD]y [PAD]a[PAD]chear[PAD] [PAD],about words are added to the dictionary each year
3,[PAD]h[PAD]in[PAD]d[PAD]i[PAD] [PAD]is [PAD]w[PAD]i[PAD]dely s[PAD]p[PAD]o[PAD]k[PAD]en[PAD] and [PAD]un[PAD]ders[PAD]to[PAD]od [PAD]b[PAD]y [PAD]g[PAD]o[PAD]an[PAD]ts[PAD] [PAD]at the s[PAD]ame [PAD]t[PAD]ime [PAD],hindi is widely spoken and understood by goans at the same time
4,b[PAD]ut[PAD] [PAD]i re[PAD]m[PAD]em[PAD]b[PAD]er[PAD] [PAD]pw[PAD]e[PAD]t[PAD]and[PAD] [PAD]p[PAD]l[PAD]aer[PAD]i[PAD]te[PAD] [PAD]w[PAD]ili[PAD]um[PAD] [PAD]sh[PAD]akes[PAD]p[PAD]eare[PAD]s[PAD] [PAD],but i remember poet and playwright william shakespeares
5,[PAD]m[PAD]a[PAD]st[PAD] [PAD]p[PAD]a[PAD]t[PAD]i[PAD]t[PAD]o[PAD] is[PAD] [PAD]m[PAD]y [PAD]w[PAD]oa[PAD]t[PAD]o[PAD] [PAD]m[PAD]il[PAD]l [PAD]wh[PAD]en[PAD] [PAD]i am[PAD] [PAD]s[PAD]h[PAD]or[PAD]d on[PAD] [PAD]t[PAD]ime [PAD]or[PAD] [PAD]in[PAD]gr[PAD]ea[PAD]d[PAD]ian[PAD] [PAD],mashed potato is my goto meal when i am short on time or ingredients
6,[PAD]are there [PAD]any[PAD] [PAD]ea[PAD]s[PAD] [PAD]e[PAD]s[PAD]s[PAD]i[PAD]sc[PAD]en[PAD]t[PAD]er[PAD] [PAD]t[PAD]ype[PAD] [PAD]p[PAD]m[PAD] [PAD]j[PAD]e[PAD]a[PAD]v[PAD]iy[PAD] [PAD]n[PAD]eare [PAD]the [PAD]are[PAD]a[PAD] [PAD]w[PAD]ith [PAD]p[PAD]in[PAD]c[PAD]o[PAD]de[PAD] [PAD] [PAD] [PAD] [PAD] [PAD]o[PAD] [PAD],are there any ecsi center type pmjay near the area with pin code
7,[PAD]which are [PAD]ti[PAD]p[PAD]i[PAD]c[PAD]al[PAD] [PAD]in m[PAD]y [PAD]re[PAD]gion[PAD]al l[PAD]angu[PAD]age[PAD] [PAD]that [PAD]is s[PAD]c[PAD]art[PAD] hea[PAD]v[PAD]ar[PAD]d[PAD]i[PAD] [PAD],which are typically in my regional language that is kathiawadi
8,[PAD]z[PAD]er[PAD]o[PAD] [PAD],zero
9,[PAD]and [PAD]there [PAD]are [PAD]s[PAD]o[PAD] [PAD]man[PAD]y[PAD] [PAD]im[PAD] [PAD]es[PAD]c[PAD]ol[PAD]le[PAD]g[PAD]e[PAD]s[PAD] [PAD]un[PAD]there the m[PAD]us[PAD]l[PAD]e[PAD]m[PAD] [PAD]g[PAD]overnments[PAD] [PAD]and[PAD] [PAD]there are s[PAD]o[PAD] m[PAD]an[PAD]y[PAD] [PAD],and there are so many mes colleges under the muslim governance and there are so many


It becomes clear that the predicted transcriptions are acoustically very similar to the target transcriptions, but often contain spelling or grammatical errors. This shouldn't be very surprising though given that we purely rely on Wav2Vec2 without making use of a language model.

In [ ]:
csv_dataset

DatasetDict({
    train: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 2516
    })
    test: Dataset({
        features: ['input_values', 'input_length', 'labels'],
        num_rows: 1336
    })
})

In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(csv_dataset["train"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] l l r e \n \n e a a h u u a a f f a a x \n \n \n [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] f f [PAD] [PAD] [PAD] v v [PAD] \r \r \r [PAD] f f f z z z a a [PAD] [PAD] [PAD] u u u r r [PAD] h h y y y [PAD] l l l [PAD] a a [PAD] [PAD] [PAD] h m m [PAD] a a l r r e a a g g [PAD] e e e m m [PAD] [PAD] [PAD] l l [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] f z z z [PAD] h h [PAD] [PAD] [PAD] [PAD] l l [PAD] [PAD] d d d [PAD] [PAD] [PAD] a [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'